In [ ]:
!pip3 install git+https://github.com/openai/whisper.git

In [ ]:
!pip3 install openai

In [22]:
import os
import whisper
import moviepy
from moviepy import VideoFileClip
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from openai import OpenAI

In [27]:

#place of video on device
video_input_path= 'video.mp4'
#folder where sound clips can be saved
audio_output_path= './audio/'
#In seconds
length_of_clip = 5
#transcript save location
text_path = 'text_from_video.txt'

In [17]:
print(number_of_clips)

125


In [28]:
#reading in video, formatting for model use (25MB or smaller clips neccesary)

clip = VideoFileClip(video_input_path)
duration = clip.duration
number_of_clips= int(duration/length_of_clip)+1


number_of_clips=3


for i in range(number_of_clips):

    start_of_clip= i*length_of_clip-2
    end_of_clip= i*length_of_clip+length_of_clip+2

    if start_of_clip<0:
        start_of_clip=0
    if end_of_clip > duration:
        end_of_clip=duration
    
    str_i = "%03d" % i
    

    subclip= clip.subclipped(start_of_clip,end_of_clip)
    subclip_path= audio_output_path+str_i+'.mp3'
    subclip.audio.write_audiofile(
        subclip_path,
        bitrate = "128k" # or it could be "192k" or "320k"
        )

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'creation_time': '2022-01-26T14:49:30.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 34, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'creation_time': '2022-01-26T14:49:30.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 01/26/2022.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'creation_time': '2022-01-26T14:49:30.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 01/26/2022.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 1243.08, 'bitrate': 166, 'start': 0.0, 'default_video_input

MoviePy - Done.
MoviePy - Writing audio in ./audio/001.mp3


MoviePy - Done.
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'creation_time': '2022-01-26T14:49:30.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 34, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'creation_time': '2022-01-26T14:49:30.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 01/26/2022.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'creation_time': '2022-01-26T14:49:30.000000Z', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 01/26/2022.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 1243.08, 'bitrate': 166, 'start': 0.0, 'def

MoviePy - Done.


In [29]:
file_list=os.listdir(audio_output_path)
file_list.sort()

In [32]:



model = whisper.load_model("turbo")
text = ''

for name in file_list:
    if not name.startswith('.') and os.path.isfile(os.path.join(audio_output_path, name)):
        # load audio and pad/trim it to fit 30 seconds
        audio = whisper.load_audio(audio_output_path+name)
        audio = whisper.pad_or_trim(audio)

        # make log-Mel spectrogram and move to the same device as the model
        mel = whisper.log_mel_spectrogram(audio=audio, n_mels=128).to(model.device)

        # detect the spoken language
        _, probs = model.detect_language(mel)
        print(f"Detected language: {max(probs, key=probs.get)}, Clip: {name}")

        # decode the audio
        options = whisper.DecodingOptions()
        result = whisper.decode(model, mel, options)

        # print the recognized text
        text += '\n'
        text += result.text
        print(result.text)

#with open(text_path, mode='wt') as f:
    #f.write(text)

/Users/lszlpotyondi/miniconda3/lib/python3.9/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=de

Detected language: hu, Clip: 000.mp3
Ebben a videóban egy viszonylag friss cikkre fogok reagálni, szoftverfejlesztő kezdő fizikusokkal.
Detected language: hu, Clip: 001.mp3
Cikkre fogok reagálni, szoftverfejlesztő kezdő fizetésekkel kapcsolatban.
Detected language: tr, Clip: 002.mp3
Müzik


In [ ]:
f= open(text_path,'r')

input_text= f.read()
f.close()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents([input_text])

In [ ]:
import chromadb.utils.embedding_functions as embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key="YOUR_API_KEY",
                model_name="text-embedding-3-small"
            )

In [ ]:
vector_db = Chroma.from_documents(
    documents=texts,
    embedding=openai_ef,
    collection_name="local-rag"
)
print("Vector database created successfully")

In [ ]:
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)